**Getting necessary libraries**

In [31]:
!pip install fake_useragent

In [32]:
import requests 
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import random
from fake_useragent import UserAgent

In [33]:
import sys
print(sys.version)


3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]


**Against Permissions accessing**

In [34]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track request
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "no-cache",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1"
}

**Important Links**
- Points Table Page: https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/points-table-standings
- All IPL Seasons Page: https://www.espncricinfo.com/records/trophy/team-match-results-season/indian-premier-league-117
- IPL T20 Website: https://www.iplt20.com/teams
- Complete Aution players: https://www.espncricinfo.com/auction/ipl-2025-auction-1460972

## 0: Teams Information

In [35]:
# dim_teams: `team_code`, `team`, `team_logo`, `captain`, `captain_img`

# -----------------> 2025 Data ------------------------->


# 1: short codes
ipl_teams_names = {
    "CSK": "Chennai Super Kings",
    "DC": "Delhi Capitals",
    "GT": "Gujarat Titans",
    "KKR": "Kolkata Knight Riders",
    "LSG": "Lucknow Super Giants",
    "MI": "Mumbai Indians",
    "PBKS": "Punjab Kings",
    "RR": "Rajasthan Royals",
    "RCB": "Royal Challengers Bengaluru",
    "SRH": "Sunrisers Hyderabad"
}




# 2: team logos
ipl_teams_logos = {
    'KKR': 'https://documents.iplt20.com/ipl/KKR/Logos/Logooutline/KKRoutline.png',
    'SRH': 'https://documents.iplt20.com/ipl/SRH/Logos/Logooutline/SRHoutline.png',
    'RR': 'https://documents.iplt20.com/ipl/RR/Logos/Logooutline/RRoutline.png',
    'RCB': 'https://documents.iplt20.com/ipl/RCB/Logos/Logooutline/RCBoutline.png',
    'CSK': 'https://documents.iplt20.com/ipl/CSK/logos/Logooutline/CSKoutline.png',
    'DC': 'https://documents.iplt20.com/ipl/DC/Logos/LogoOutline/DCoutline.png',
    'LSG': 'https://documents.iplt20.com/ipl/LSG/Logos/Logooutline/LSGoutline.png',
    'GT': 'https://documents.iplt20.com/ipl/GT/Logos/Logooutline/GToutline.png',
    'PBKS': 'https://documents.iplt20.com/ipl/PBKS/Logos/Logooutline/PBKSoutline.png', 
    'MI': 'https://documents.iplt20.com/ipl/MI/Logos/Logooutline/MIoutline.png'
}




# create dataframe
df_ipl_teams = pd.DataFrame(list(ipl_teams_names.items()), columns=["team_code", "team"])

# add captain name, logo and other info
df_ipl_teams['team_logo'] = df_ipl_teams['team_code'].map(ipl_teams_logos)

print(df_ipl_teams.shape)
df_ipl_teams.head(10)

(10, 3)


,team_code,team,team_logo
0,CSK,Chennai Super Kings,https://documents.iplt20.com/ipl/CSK/logos/Log...
1,DC,Delhi Capitals,https://documents.iplt20.com/ipl/DC/Logos/Logo...
2,GT,Gujarat Titans,https://documents.iplt20.com/ipl/GT/Logos/Logo...
3,KKR,Kolkata Knight Riders,https://documents.iplt20.com/ipl/KKR/Logos/Log...
4,LSG,Lucknow Super Giants,https://documents.iplt20.com/ipl/LSG/Logos/Log...
5,MI,Mumbai Indians,https://documents.iplt20.com/ipl/MI/Logos/Logo...
6,PBKS,Punjab Kings,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...
7,RR,Rajasthan Royals,https://documents.iplt20.com/ipl/RR/Logos/Logo...
8,RCB,Royal Challengers Bengaluru,https://documents.iplt20.com/ipl/RCB/Logos/Log...
9,SRH,Sunrisers Hyderabad,https://documents.iplt20.com/ipl/SRH/Logos/Log...


## 1: Points Table

In [36]:
# Step 1: Send a GET request to the website
url = 'https://www.espncricinfo.com/series/ipl-2025-1449924/points-table-standings'
response = requests.get(url, headers=headers)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# step3: Get the table
tables = soup.select('div > table.ds-table')[0]

# step4: Iterate through each row
allRows = soup.select('table > tbody > tr.ds-text-tight-s')
pointsTable = []

for element in allRows:
    tds = element.find_all('td')
     
    team = re.sub(r'\d+', '', tds[0].text.strip())
    total_matches_played = tds[1].text.strip()
    total_matches_won = tds[2].text.strip()
    total_matches_loss = tds[3].text.strip()
    total_matches_tied = tds[4].text.strip()
    total_matches_abonded = tds[5].text.strip()
    points_awarded = tds[6].text.strip()
    net_run_rate = tds[7].text.strip()
    
    pointsTable.append({
        'team': team,
        'matches_played': total_matches_played,
        'won': total_matches_won,
        'lost': total_matches_loss,
        'tied': total_matches_tied,
        'abonded': total_matches_abonded,
        'points': points_awarded,
        "net_run_rate": net_run_rate
    })


# Step 5: Convert the data into a pandas DataFrame
df_points_table = pd.DataFrame(pointsTable)
# let's add short code of team
ipl_teams = {
    'Kolkata Knight Riders': 'KKR',
    'Sunrisers Hyderabad': 'SRH',
    'Rajasthan Royals': 'RR',
    'Royal Challengers Bengaluru': 'RCB',
    'Chennai Super Kings': 'CSK',
    'Delhi Capitals': 'DC',
    'Lucknow Super Giants': 'LSG',
    'Gujarat Titans': 'GT',
    'Punjab Kings': 'PBKS',
    'Mumbai Indians': 'MI'
}
df_points_table['team_code'] = df_points_table['team'].map(ipl_teams)

#re-arrange
cols = ['team_code', 'team', 'matches_played', 'won', 'lost', 'tied', 'abonded', 'points', 'net_run_rate']
df_points_table = df_points_table[cols]

print(df_points_table.shape)
df_points_table.head(10)


(10, 9)


,team_code,team,matches_played,won,lost,tied,abonded,points,net_run_rate
0,GT,Gujarat Titans,5,4,1,0,0,8,1.413
1,DC,Delhi Capitals,4,4,0,0,0,8,1.278
2,RCB,Royal Challengers Bengaluru,5,3,2,0,0,6,0.539
3,PBKS,Punjab Kings,4,3,1,0,0,6,0.289
4,LSG,Lucknow Super Giants,5,3,2,0,0,6,0.078
5,KKR,Kolkata Knight Riders,5,2,3,0,0,4,-0.056
6,RR,Rajasthan Royals,5,2,3,0,0,4,-0.733
7,MI,Mumbai Indians,5,1,4,0,0,2,-0.010
8,CSK,Chennai Super Kings,5,1,4,0,0,2,-0.889
9,SRH,Sunrisers Hyderabad,5,1,4,0,0,2,-1.629


In [37]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

## 2: Match Summary

In [38]:
ua = UserAgent()
headers = {
    "User-Agent": ua.random,
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1"
}



# Step 1: Send a GET request to the website
url = 'https://www.espncricinfo.com/records/season/team-match-results/2025-2025?trophy=117'
response = requests.get(url, headers=headers)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Find the table rows and extract the data from the cells
allRows = soup.select('table > tbody > tr')
matchSummary = []

for element in allRows:
    tds = element.find_all('td')

    if tds[0].text.strip() == 'Punjab Kings':
        team1 = 'PBKS'
    else:
        team1 = tds[0].text.strip()
        
    if tds[1].text.strip() == 'Punjab Kings':
        team2 = 'PBKS'
    else:
        team2 = tds[1].text.strip()

    if tds[2].text.strip() == 'Punjab Kings':
        winner = 'PBKS'
    else:
        winner = tds[2].text.strip()
        
    margin = tds[3].text.strip()
    ground = tds[4].text.strip()
    matchDate = tds[5].text.strip()
    scorecard_link = "https://www.espncricinfo.com" + tds[6].find('a')['href']

    matchSummary.append({
        'team1': team1,
        'team2': team2,
        'winner': winner,
        'margin': margin,
        'ground': ground,
        'matchDate': matchDate,
        'scorecard_link': scorecard_link
    })

# Step 4: Creating a new list to filter out matches with a valid margin
newMatchSummary = []

for match in matchSummary:
    if match['margin'] != '-':
        if ('wickets' in match['margin']) or ('wicket' in match['margin']):
            finT2 = match['winner']
            finT1 = match['team1'] if match['team1'] != match['winner'] else match['team2']
        elif ('runs' in match['margin']) or ('run' in match['margin']):
            finT1 = match['winner']
            finT2 = match['team1'] if match['team1'] != match['winner'] else match['team2']

        newMatchSummary.append({
            'match': f'{finT1} Vs {finT2}',
            'team1': finT1,
            'team2': finT2,
            'winner': match['winner'],
            'margin': match['margin'],
            'matchDate': match['matchDate'],
            'scorecard_link': match['scorecard_link']
        })

# Step 5: Convert the data into a pandas DataFrame
df_match_summary = pd.DataFrame(newMatchSummary)


# Step6: Let's add match ID
df_match_summary['combined_cols'] = df_match_summary['team1'] + ' Vs ' + df_match_summary['team2'] + ' Vs ' + df_match_summary['matchDate']

#create a list of all unique values 
all_matches = df_match_summary['combined_cols'].unique().tolist()

#let assign a unique id for each match_id 
unique_ids = set()
while len(unique_ids) < len(all_matches):
    unique_id = "T20" + str(random.randint(1000, 9999))
    unique_ids.add(unique_id)
unique_ids_list = list(unique_ids)

# create a dictionary for this
dict_matches = {}
for match_combined, match_id in zip(all_matches, unique_ids):
    dict_matches[match_combined] = match_id

# add new column
df_match_summary['matchID'] = df_match_summary['combined_cols'].map(dict_matches)

# drop combined cols
df_match_summary.drop("combined_cols", axis = 1, inplace = True)

# re-arrange cols
cols = ['matchID', 'matchDate', 'match', 'team1', 'team2', 'winner', 'margin', 'scorecard_link']
df_match_summary = df_match_summary[cols]

print(df_match_summary.shape)
df_match_summary.head()

(24, 8)


,matchID,matchDate,match,team1,team2,winner,margin,scorecard_link
0,T201958,"Apr 10, 2025",RCB Vs DC,RCB,DC,DC,6 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
1,T202205,"Apr 9, 2025",GT Vs RR,GT,RR,GT,58 runs,https://www.espncricinfo.com/series/ipl-2025-1...
2,T209361,"Apr 8, 2025",PBKS Vs CSK,PBKS,CSK,PBKS,18 runs,https://www.espncricinfo.com/series/ipl-2025-1...
3,T202678,"Apr 8, 2025",LSG Vs KKR,LSG,KKR,LSG,4 runs,https://www.espncricinfo.com/series/ipl-2025-1...
4,T204077,"Apr 7, 2025",RCB Vs MI,RCB,MI,RCB,12 runs,https://www.espncricinfo.com/series/ipl-2025-1...


In [39]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

## 3: Batting | Bowling Summary + Players Links

In [40]:
# define the lists
batting_summary = []
bowling_summary = []
extra_match_info = []

# Step 1: Iterate through each row in the dataframe
for _, row in df_match_summary.iterrows():

    #list to get all team players (To get captain information)
    team1_players = []
    team2_players = []
    
    matchID = row['matchID']
    team1 = row['team1']
    team2 = row['team2']
    match_info = row['match']
    scorecard_link = row['scorecard_link']
    matchDate = row['matchDate']

    #2: Navigate to scorecard link
    response = requests.get(scorecard_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    #3: Get the batting information tables
    batting_tables = soup.select('div > table.ci-scorecard-table')
    first_inning_batting_rows = [row for row in batting_tables[0].select('tbody > tr') if len(row.select('td')) >= 8]
    second_inning_batting_rows = [row for row in batting_tables[1].select('tbody > tr') if len(row.select('td')) >= 8]


    # 4: Get the bowling information tables
    bowling_tables = soup.select('div > table.ds-table')
    first_inning_bowling_rows = [row for row in bowling_tables[1].select('tbody > tr') if len(row.select('td')) >= 11]
    second_inning_bowling_rows = [row for row in bowling_tables[3].select('tbody > tr') if len(row.select('td')) >= 11]

    # 5: Get the batting records
    #First Innings
    for i in range(0, len(first_inning_batting_rows)):
        element = first_inning_batting_rows[i]
        index = i
    
        tds = element.find_all('td')
        
        #for dismal (handling specially)
        if tds[1].find('span') is not None:
            dismissal =  tds[1].find('span').find('span').text
        else:
            dismissal = tds[1].text.strip()

        batting_summary.append({
            "matchID": matchID,
            "match": match_info,
            "teamInnings": team1,
            "battingPos": index + 1,
            "batsmanName": tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''),
            "dismissal": dismissal,
            "runs": tds[2].find('strong').text,
            "balls": tds[3].text,
            "4s": tds[5].text,
            "6s": tds[6].text,
            "SR": tds[7].text,
            "Innings": "1st Innings"
        })
        
        team1_players.append(tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', '')) 
    
    #Second Innings
    for i in range(0, len(second_inning_batting_rows)):
        element = second_inning_batting_rows[i]
        index = i
    
        tds = element.find_all('td')
        
        #for dismal (handling specially)
        if tds[1].find('span') is not None:
            dismissal =  tds[1].find('span').find('span').text
        else:
            dismissal = tds[1].text.strip()

        batting_summary.append({
            "matchID": matchID,
            "match": match_info,
            "teamInnings": team2,
            "battingPos": index + 1,
            "batsmanName": tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''),
            "dismissal": dismissal,
            "runs": tds[2].find('strong').text,
            "balls": tds[3].text,
            "4s": tds[5].text,
            "6s": tds[6].text,
            "SR": tds[7].text,
            "Innings": "2nd Innings"
        })

        team2_players.append(tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''))



    # did not bat (team1 players)
    try:
        did_not_bat_players = soup.select('div.ds-text-tight-m.ds-font-regular.ds-leading-4.ds-text-typo-mid1')

        if len(did_not_bat_players) == 2:
            # team1
            team1_did_not_bat = did_not_bat_players[0]
            all_spans_team1 = team1_did_not_bat.find_all('span')
            team1_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team1]))
            team1_players = team1_players + team1_span_texts
    
            # team2
            team2_did_not_bat = did_not_bat_players[1]
            all_spans_team2 = team2_did_not_bat.find_all('span')
            team2_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team2]))
            team2_players = team2_players + team2_span_texts

        elif len(did_not_bat_players) == 1:
            if len(team1_players) != 11:
                team1_did_not_bat = did_not_bat_players[0]
                all_spans_team1 = team1_did_not_bat.find_all('span')
                team1_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team1]))
                team1_players = team1_players + team1_span_texts
            else:
                team2_did_not_bat = did_not_bat_players[0]
                all_spans_team2 = team2_did_not_bat.find_all('span')
                team2_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team2]))
                team2_players = team2_players + team2_span_texts
    except Exception:
        pass  # Ignore all exceptions silently


    # 6: Get the bowling records
    #First Innings
    for index, element in enumerate(first_inning_bowling_rows):
            tds = element.find_all('td')
            bowling_summary.append({
                "matchID": matchID,
                "match": match_info,
                "bowlingTeam": team2,
                "bowlerName": tds[0].find('a').find('span').text.replace('â€', '').replace('\xa0', ''),
                "overs": tds[1].text,
                "maiden": tds[2].text,
                "runs": tds[3].text,
                "wickets": tds[4].text,
                "economy": tds[5].text,
                "0s": tds[6].text,
                "4s": tds[7].text,
                "6s": tds[8].text,
                "wides": tds[9].text,
                "noBalls": tds[10].text,
                "Innings": "1st Innings"
            })


    #Second Innings
    for index, element in enumerate(second_inning_bowling_rows):
        tds = element.find_all('td')
        bowling_summary.append({
            "matchID": matchID,
            "match": match_info,
            "bowlingTeam": team1,
            "bowlerName": tds[0].find('a').find('span').text.replace('â€', '').replace('\xa0', ''),
            "overs": tds[1].text,
            "maiden": tds[2].text,
            "runs": tds[3].text,
            "wickets": tds[4].text,
            "economy": tds[5].text,
            "0s": tds[6].text,
            "4s": tds[7].text,
            "6s": tds[8].text,
            "wides": tds[9].text,
            "noBalls": tds[10].text,
            "Innings": "2nd Innings"
        })


    # 7: Get Extra match info
    team_scores = soup.select('div.ds-mb-1 > div.ci-team-score')
    team1_ex = team_scores[0].select('div')[0].text
    team1_score = team_scores[0].select('div')[1].text
    team2_ex = team_scores[1].select('div')[0].text
    team2_score = team_scores[1].select('div')[1].text
    result_text = soup.select('p.ds-text-tight-s.ds-font-medium.ds-truncate.ds-text-typo')[0].text
    
    # Locate the span with "RESULT"
    result_span = soup.find('span', string="RESULT")
    if result_span:
    # Go up to the parent and find the div that has the match text
        match_info_div = result_span.find_parent().find_next("div", class_="ds-text-tight-xs ds-truncate ds-text-typo-mid3 ds-mb-1")
        if match_info_div:
            match_location_date = match_info_div.get_text(strip=True)
    #     else:
    #         print("❌ Match info div not found.")
    # else:
    #     print("❌ RESULT span not found.")


    # toss and player of match
    k_table = soup.select('div.ds-p-0 > table.ds-w-full.ds-table.ds-table-sm.ds-table-auto')[0]
    toss = k_table.select('tr')[1].text.replace('Toss', '')

    if 'Player Of The Match' in k_table.select('tr')[4].text:
        player_of_the_match = k_table.select('tr')[4].text.replace('Player Of The Match', '').replace('â€', '').replace('\xa0', '').replace("(c)", "").replace('†', '')
    else:
        player_of_the_match = k_table.select('tr')[5].text.replace('Player Of The Match', '').replace('â€', '').replace('\xa0', '').replace("(c)", "").replace('†', '')
    

    #team1 captain
    if '(c)' in team1_players:
        team1_players.remove('(c)')
    team1_captain = [player for player in team1_players if "(c)" in player]
    team1_captain_name = team1_captain[0].replace("(c)", "").replace('†', '').strip() if team1_captain else None

    #team2 captain
    if '(c)' in team2_players:
        team2_players.remove('(c)')
    team2_captain = [player for player in team2_players if "(c)" in player]
    team2_captain_name = team2_captain[0].replace("(c)", "").replace('†', '').strip() if team2_captain else None

    extra_match_info.append({
            "matchID": matchID,
            "team1": team1_ex,
            "team1_captain": team1_captain_name,
            "team1_score":  team1_score,
            "team2": team2_ex,
            "team2_captain": team2_captain_name,
            "team2_score": team2_score, 
            "result": result_text,
            "match_location_date": match_location_date,
            "toss": toss,
            "player_of_the_match": player_of_the_match
        })

#creating individual dataframes
batting_summary_df = pd.DataFrame(batting_summary)
bowling_summary_df = pd.DataFrame(bowling_summary)
extra_match_info = pd.DataFrame(extra_match_info)



# data cleaning
#batting
batting_summary_df['batsmanName'] = batting_summary_df['batsmanName'].apply(lambda x: x.replace('(c)', ''))
batting_summary_df['batsmanName']= batting_summary_df['batsmanName'].apply(lambda x: x.replace('†', ''))
batting_summary_df['out/not_out'] = batting_summary_df.dismissal.apply(lambda x: "out" if x != "not out" else "not_out")
batting_summary_df.drop(columns=["dismissal"], inplace=True)

#bowling
bowling_summary_df['bowlerName'] = bowling_summary_df['bowlerName'].apply(lambda x: x.replace('(c)', ''))
bowling_summary_df['bowlerName'] = bowling_summary_df['bowlerName'].apply(lambda x: x.replace('†', ''))

In [41]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

**Batting**

In [42]:
print(batting_summary_df.shape)
batting_summary_df.head(5)

(393, 12)


,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
0,T201958,RCB Vs DC,RCB,1,Phil Salt,37,17,4,3,217.64,1st Innings,out
1,T201958,RCB Vs DC,RCB,2,Virat Kohli,22,14,1,2,157.14,1st Innings,out
2,T201958,RCB Vs DC,RCB,3,Devdutt Padikkal,1,8,0,0,12.50,1st Innings,out
3,T201958,RCB Vs DC,RCB,4,Rajat Patidar,25,23,1,1,108.69,1st Innings,out
4,T201958,RCB Vs DC,RCB,5,Liam Livingstone,4,6,0,0,66.66,1st Innings,out


In [43]:
batting_summary_df.tail(5)

,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
388,T204727,KKR Vs RCB,RCB,1,Phil Salt,56,31,9,2,180.64,2nd Innings,out
389,T204727,KKR Vs RCB,RCB,2,Virat Kohli,59,36,4,3,163.88,2nd Innings,not_out
390,T204727,KKR Vs RCB,RCB,3,Devdutt Padikkal,10,10,1,0,100.00,2nd Innings,out
391,T204727,KKR Vs RCB,RCB,4,Rajat Patidar,34,16,5,1,212.50,2nd Innings,out
392,T204727,KKR Vs RCB,RCB,5,Liam Livingstone,15,5,2,1,300.00,2nd Innings,not_out


**Bowling**

In [44]:
print(bowling_summary_df.shape)
bowling_summary_df.head(6)

(286, 15)


,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
0,T201958,RCB Vs DC,DC,Mitchell Starc,3,0,35,0,11.66,9,3,2,1,1,1st Innings
1,T201958,RCB Vs DC,DC,Axar Patel,4,0,52,0,13.00,8,3,5,1,0,1st Innings
2,T201958,RCB Vs DC,DC,Vipraj Nigam,4,0,18,2,4.50,12,0,1,0,0,1st Innings
3,T201958,RCB Vs DC,DC,Mukesh Kumar,3,1,26,1,8.66,10,1,2,3,0,1st Innings
4,T201958,RCB Vs DC,DC,Kuldeep Yadav,4,0,17,2,4.25,11,1,0,0,0,1st Innings
5,T201958,RCB Vs DC,DC,Mohit Sharma,2,0,10,1,5.00,5,1,0,0,0,1st Innings


In [45]:
bowling_summary_df.tail(6)

,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
280,T204727,KKR Vs RCB,RCB,Liam Livingstone,2,0,14,0,7.00,2,1,0,0,0,1st Innings
281,T204727,KKR Vs RCB,KKR,Vaibhav Arora,3,0,42,1,14.00,4,7,1,1,0,2nd Innings
282,T204727,KKR Vs RCB,KKR,Spencer Johnson,2.2,0,31,0,13.28,7,3,3,0,0,2nd Innings
283,T204727,KKR Vs RCB,KKR,Varun Chakravarthy,4,0,43,1,10.75,7,5,2,0,0,2nd Innings
284,T204727,KKR Vs RCB,KKR,Harshit Rana,3,0,32,0,10.66,4,5,0,0,0,2nd Innings
285,T204727,KKR Vs RCB,KKR,Sunil Narine,4,0,27,1,6.75,5,1,1,0,0,2nd Innings


**Extra Match Info**

In [46]:
print(extra_match_info.shape)
extra_match_info.head()

(24, 11)


,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
0,T201958,RCB,Rajat Patidar,163/7,DC,Axar Patel,"(17.5/20 ov, T:164) 169/4",DC won by 6 wickets (with 13 balls remaining),"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",KL Rahul
1,T202205,GT,Shubman Gill,217/6,RR,Sanju Samson,"(19.2/20 ov, T:218) 159",GT won by 58 runs,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",Sai Sudharsan
2,T209361,PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,"(20 ov, T:220) 201/5",PBKS won by 18 runs,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",Priyansh Arya
3,T202678,LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,"(20 ov, T:239) 234/7",LSG won by 4 runs,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",Nicholas Pooran
4,T204077,RCB,Rajat Patidar,221/5,MI,Hardik Pandya,"(20 ov, T:222) 209/9",RCB won by 12 runs,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",Rajat Patidar


In [47]:
extra_match_info.tail()

,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
19,T208793,PBKS,Shreyas Iyer,243/5,GT,Shubman Gill,"(20 ov, T:244) 232/5",PBKS won by 11 runs,"5th Match (N), Ahmedabad, March 25, 2025,India...","Gujarat Titans, elected to field first",Shreyas Iyer
20,T209611,LSG,Rishabh Pant,209/8,DC,Axar Patel,"(19.3/20 ov, T:210) 211/9",DC won by 1 wicket (with 3 balls remaining),"4th Match (N), Visakhapatnam, March 24, 2025,I...","Delhi Capitals, elected to field first",Ashutosh Sharma
21,T201022,MI,Suryakumar Yadav,155/9,CSK,Ruturaj Gaikwad,"(19.1/20 ov, T:156) 158/6",CSK won by 4 wickets (with 5 balls remaining),"3rd Match (N), Chennai, March 23, 2025,Indian ...","Chennai Super Kings, elected to field first",Noor Ahmad
22,T203130,SRH,Pat Cummins,286/6,RR,Riyan Parag,"(20 ov, T:287) 242/6",SRH won by 44 runs,"2nd Match (D/N), Hyderabad, March 23, 2025,Ind...","Rajasthan Royals, elected to field first",Ishan Kishan
23,T204727,KKR,Ajinkya Rahane,174/8,RCB,Rajat Patidar,"(16.2/20 ov, T:175) 177/3",RCB won by 7 wickets (with 22 balls remaining),"1st Match (N), Eden Gardens, March 22, 2025,In...","Royal Challengers Bengaluru, elected to field ...",Krunal Pandya


## 4: Processing Match and Extra Match info

In [48]:
print(extra_match_info.shape)
extra_match_info.head()

(24, 11)


,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
0,T201958,RCB,Rajat Patidar,163/7,DC,Axar Patel,"(17.5/20 ov, T:164) 169/4",DC won by 6 wickets (with 13 balls remaining),"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",KL Rahul
1,T202205,GT,Shubman Gill,217/6,RR,Sanju Samson,"(19.2/20 ov, T:218) 159",GT won by 58 runs,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",Sai Sudharsan
2,T209361,PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,"(20 ov, T:220) 201/5",PBKS won by 18 runs,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",Priyansh Arya
3,T202678,LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,"(20 ov, T:239) 234/7",LSG won by 4 runs,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",Nicholas Pooran
4,T204077,RCB,Rajat Patidar,221/5,MI,Hardik Pandya,"(20 ov, T:222) 209/9",RCB won by 12 runs,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",Rajat Patidar


In [50]:
print(df_match_summary.shape)
df_match_summary.head()

(24, 8)


,matchID,matchDate,match,team1,team2,winner,margin,scorecard_link
0,T201958,"Apr 10, 2025",RCB Vs DC,RCB,DC,DC,6 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
1,T202205,"Apr 9, 2025",GT Vs RR,GT,RR,GT,58 runs,https://www.espncricinfo.com/series/ipl-2025-1...
2,T209361,"Apr 8, 2025",PBKS Vs CSK,PBKS,CSK,PBKS,18 runs,https://www.espncricinfo.com/series/ipl-2025-1...
3,T202678,"Apr 8, 2025",LSG Vs KKR,LSG,KKR,LSG,4 runs,https://www.espncricinfo.com/series/ipl-2025-1...
4,T204077,"Apr 7, 2025",RCB Vs MI,RCB,MI,RCB,12 runs,https://www.espncricinfo.com/series/ipl-2025-1...


In [51]:
### Merge both the columns

df_final_match_summary = df_match_summary.merge(extra_match_info, on = 'matchID')

req_cols = ['matchID', 'matchDate', 'match', 'match_location_date', 'toss', 'team1_x', 'team1_captain', 'team1_score', 
            'team2_x', 'team2_captain', 'team2_score', 'winner', 'result', 'player_of_the_match']

df_final_match_summary = df_final_match_summary[req_cols]

#Rename cols
rename_cols = {'team1_x': 'team1', 'team2_x': 'team2'}
df_final_match_summary.rename(rename_cols, axis = 1, inplace = True)

print(df_final_match_summary.shape)
df_final_match_summary.head()

(24, 14)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,team2_score,winner,result,player_of_the_match
0,T201958,"Apr 10, 2025",RCB Vs DC,"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",RCB,Rajat Patidar,163/7,DC,Axar Patel,"(17.5/20 ov, T:164) 169/4",DC,DC won by 6 wickets (with 13 balls remaining),KL Rahul
1,T202205,"Apr 9, 2025",GT Vs RR,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",GT,Shubman Gill,217/6,RR,Sanju Samson,"(19.2/20 ov, T:218) 159",GT,GT won by 58 runs,Sai Sudharsan
2,T209361,"Apr 8, 2025",PBKS Vs CSK,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,"(20 ov, T:220) 201/5",PBKS,PBKS won by 18 runs,Priyansh Arya
3,T202678,"Apr 8, 2025",LSG Vs KKR,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,"(20 ov, T:239) 234/7",LSG,LSG won by 4 runs,Nicholas Pooran
4,T204077,"Apr 7, 2025",RCB Vs MI,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",RCB,Rajat Patidar,221/5,MI,Hardik Pandya,"(20 ov, T:222) 209/9",RCB,RCB won by 12 runs,Rajat Patidar


## 5: Players Data

In [52]:
players_info_df_final = pd.read_csv('abfss://1e8640c6-dc84-47a8-b295-452f8a310af8@onelake.dfs.fabric.microsoft.com/327dd03d-fc61-4a7c-8838-35fa63ac0dd5/Files/2025/dim_players.csv')
print(players_info_df_final.shape)
players_info_df_final.head()

(227, 7)


,name,team,link,battingStyle,bowlingStyle,playingRole,image
0,Heinrich Klaasen,SRH,https://www.espncricinfo.com//cricketers/heinr...,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,Pat Cummins,SRH,https://www.espncricinfo.com//cricketers/pat-c...,Right hand Bat,Right arm Fast,Bowler,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,Abhishek Sharma,SRH,https://www.espncricinfo.com//cricketers/abhis...,Left hand Bat,Slow Left arm Orthodox,Batting Allrounder,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,Travis Head,SRH,https://www.espncricinfo.com//cricketers/travi...,Left hand Bat,Right arm Offbreak,Top order Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,Ishan Kishan,SRH,https://www.espncricinfo.com//cricketers/ishan...,Left hand Bat,Legbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...


**Comparison Check**

In [53]:
all_batsman_names = batting_summary_df['batsmanName'].unique().tolist()
all_bowler_names = bowling_summary_df['bowlerName'].unique().tolist()
all_player_names = players_info_df_final['name'].unique().tolist()

print('Total Players in Batsman: ', len(all_batsman_names))
print('Total Players in Bowlers: ', len(all_bowler_names))
print('Total Players: ', len(all_player_names))

Total Players in Batsman:  121
Total Players in Bowlers:  88
Total Players:  227


In [54]:
#Batsman Check: (If null, then all batsman players present in dim_players)

for batsman_play in all_batsman_names:
    if batsman_play not in all_player_names:
        print(batsman_play)

In [55]:
#Bowlers Check: (If null, then all bowler players present in dim_players)

for bowl_play in all_bowler_names:
    if bowl_play not in all_player_names:
        print(bowl_play)

### Adding players images in match summary

In [56]:
print(df_final_match_summary.shape)
df_final_match_summary.head()

(24, 14)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,team2_score,winner,result,player_of_the_match
0,T201958,"Apr 10, 2025",RCB Vs DC,"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",RCB,Rajat Patidar,163/7,DC,Axar Patel,"(17.5/20 ov, T:164) 169/4",DC,DC won by 6 wickets (with 13 balls remaining),KL Rahul
1,T202205,"Apr 9, 2025",GT Vs RR,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",GT,Shubman Gill,217/6,RR,Sanju Samson,"(19.2/20 ov, T:218) 159",GT,GT won by 58 runs,Sai Sudharsan
2,T209361,"Apr 8, 2025",PBKS Vs CSK,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,"(20 ov, T:220) 201/5",PBKS,PBKS won by 18 runs,Priyansh Arya
3,T202678,"Apr 8, 2025",LSG Vs KKR,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,"(20 ov, T:239) 234/7",LSG,LSG won by 4 runs,Nicholas Pooran
4,T204077,"Apr 7, 2025",RCB Vs MI,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",RCB,Rajat Patidar,221/5,MI,Hardik Pandya,"(20 ov, T:222) 209/9",RCB,RCB won by 12 runs,Rajat Patidar


In [57]:
# new cols to add
# 1. team1_logo, team2_logo, team1_captain_logo, team2_captain_logo, player_of_match_logo


# step1: Add team1 and team2 logo and also full form of team
ipl_teams_logos = {
    'KKR': 'https://documents.iplt20.com/ipl/KKR/Logos/Logooutline/KKRoutline.png',
    'SRH': 'https://documents.iplt20.com/ipl/SRH/Logos/Logooutline/SRHoutline.png',
    'RR': 'https://documents.iplt20.com/ipl/RR/Logos/Logooutline/RRoutline.png',
    'RCB': 'https://documents.iplt20.com/ipl/RCB/Logos/Logooutline/RCBoutline.png',
    'CSK': 'https://documents.iplt20.com/ipl/CSK/logos/Logooutline/CSKoutline.png',
    'DC': 'https://documents.iplt20.com/ipl/DC/Logos/LogoOutline/DCoutline.png',
    'LSG': 'https://documents.iplt20.com/ipl/LSG/Logos/Logooutline/LSGoutline.png',
    'GT': 'https://documents.iplt20.com/ipl/GT/Logos/Logooutline/GToutline.png',
    'PBKS': 'https://documents.iplt20.com/ipl/PBKS/Logos/Logooutline/PBKSoutline.png', 
    'MI': 'https://documents.iplt20.com/ipl/MI/Logos/Logooutline/MIoutline.png'
}

ipl_teams_names = {
    "CSK": "Chennai Super Kings",
    "DC": "Delhi Capitals",
    "GT": "Gujarat Titans",
    "KKR": "Kolkata Knight Riders",
    "LSG": "Lucknow Super Giants",
    "MI": "Mumbai Indians",
    "PBKS": "Punjab Kings",
    "RR": "Rajasthan Royals",
    "RCB": "Royal Challengers Bengaluru",
    "SRH": "Sunrisers Hyderabad"
}

#logo
df_final_match_summary['team1_logo'] = df_final_match_summary['team1'].map(ipl_teams_logos)
df_final_match_summary['team2_logo'] = df_final_match_summary['team2'].map(ipl_teams_logos)

# full form
df_final_match_summary['team1_full_name'] = df_final_match_summary['team1'].map(ipl_teams_names)
df_final_match_summary['team2_full_name'] = df_final_match_summary['team2'].map(ipl_teams_names)


# step2: Add players images (team1 captain, team2 captain, player_of_match_img
# players dictionary
name_image_dict = players_info_df_final.set_index("name")["image"].to_dict()

df_final_match_summary['team1_captain_image'] = df_final_match_summary['team1_captain'].map(name_image_dict)
df_final_match_summary['team2_captain_image'] = df_final_match_summary['team2_captain'].map(name_image_dict)
df_final_match_summary['player_of_the_match_image'] = df_final_match_summary['player_of_the_match'].map(name_image_dict)

In [58]:
print(df_final_match_summary.shape)
df_final_match_summary.head()

(24, 21)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,...,winner,result,player_of_the_match,team1_logo,team2_logo,team1_full_name,team2_full_name,team1_captain_image,team2_captain_image,player_of_the_match_image
0,T201958,"Apr 10, 2025",RCB Vs DC,"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",RCB,Rajat Patidar,163/7,DC,Axar Patel,...,DC,DC won by 6 wickets (with 13 balls remaining),KL Rahul,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://documents.iplt20.com/ipl/DC/Logos/Logo...,Royal Challengers Bengaluru,Delhi Capitals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,T202205,"Apr 9, 2025",GT Vs RR,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",GT,Shubman Gill,217/6,RR,Sanju Samson,...,GT,GT won by 58 runs,Sai Sudharsan,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://documents.iplt20.com/ipl/RR/Logos/Logo...,Gujarat Titans,Rajasthan Royals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,T209361,"Apr 8, 2025",PBKS Vs CSK,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,...,PBKS,PBKS won by 18 runs,Priyansh Arya,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,https://documents.iplt20.com/ipl/CSK/logos/Log...,Punjab Kings,Chennai Super Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,T202678,"Apr 8, 2025",LSG Vs KKR,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,...,LSG,LSG won by 4 runs,Nicholas Pooran,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://documents.iplt20.com/ipl/KKR/Logos/Log...,Lucknow Super Giants,Kolkata Knight Riders,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,T204077,"Apr 7, 2025",RCB Vs MI,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",RCB,Rajat Patidar,221/5,MI,Hardik Pandya,...,RCB,RCB won by 12 runs,Rajat Patidar,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Royal Challengers Bengaluru,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...


## Data Preview

In [59]:
# teams data

print(df_ipl_teams.shape)
df_ipl_teams.head(10)

(10, 3)


,team_code,team,team_logo
0,CSK,Chennai Super Kings,https://documents.iplt20.com/ipl/CSK/logos/Log...
1,DC,Delhi Capitals,https://documents.iplt20.com/ipl/DC/Logos/Logo...
2,GT,Gujarat Titans,https://documents.iplt20.com/ipl/GT/Logos/Logo...
3,KKR,Kolkata Knight Riders,https://documents.iplt20.com/ipl/KKR/Logos/Log...
4,LSG,Lucknow Super Giants,https://documents.iplt20.com/ipl/LSG/Logos/Log...
5,MI,Mumbai Indians,https://documents.iplt20.com/ipl/MI/Logos/Logo...
6,PBKS,Punjab Kings,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...
7,RR,Rajasthan Royals,https://documents.iplt20.com/ipl/RR/Logos/Logo...
8,RCB,Royal Challengers Bengaluru,https://documents.iplt20.com/ipl/RCB/Logos/Log...
9,SRH,Sunrisers Hyderabad,https://documents.iplt20.com/ipl/SRH/Logos/Log...


In [60]:
# Points Table
print(df_points_table.shape)
df_points_table.head()

(10, 9)


,team_code,team,matches_played,won,lost,tied,abonded,points,net_run_rate
0,GT,Gujarat Titans,5,4,1,0,0,8,1.413
1,DC,Delhi Capitals,4,4,0,0,0,8,1.278
2,RCB,Royal Challengers Bengaluru,5,3,2,0,0,6,0.539
3,PBKS,Punjab Kings,4,3,1,0,0,6,0.289
4,LSG,Lucknow Super Giants,5,3,2,0,0,6,0.078


In [61]:
# Match Summary
print(df_final_match_summary.shape)
df_final_match_summary.head()

(24, 21)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,...,winner,result,player_of_the_match,team1_logo,team2_logo,team1_full_name,team2_full_name,team1_captain_image,team2_captain_image,player_of_the_match_image
0,T201958,"Apr 10, 2025",RCB Vs DC,"24th Match (N), Bengaluru, April 10, 2025,Indi...","Delhi Capitals, elected to field first",RCB,Rajat Patidar,163/7,DC,Axar Patel,...,DC,DC won by 6 wickets (with 13 balls remaining),KL Rahul,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://documents.iplt20.com/ipl/DC/Logos/Logo...,Royal Challengers Bengaluru,Delhi Capitals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,T202205,"Apr 9, 2025",GT Vs RR,"23rd Match (N), Ahmedabad, April 09, 2025,Indi...","Rajasthan Royals, elected to field first",GT,Shubman Gill,217/6,RR,Sanju Samson,...,GT,GT won by 58 runs,Sai Sudharsan,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://documents.iplt20.com/ipl/RR/Logos/Logo...,Gujarat Titans,Rajasthan Royals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,T209361,"Apr 8, 2025",PBKS Vs CSK,"22nd Match (N), Mullanpur, April 08, 2025,Indi...","Punjab Kings, elected to bat first",PBKS,Shreyas Iyer,219/6,CSK,Ruturaj Gaikwad,...,PBKS,PBKS won by 18 runs,Priyansh Arya,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,https://documents.iplt20.com/ipl/CSK/logos/Log...,Punjab Kings,Chennai Super Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,T202678,"Apr 8, 2025",LSG Vs KKR,"21st Match (D/N), Eden Gardens, April 08, 2025...","Kolkata Knight Riders, elected to field first",LSG,Rishabh Pant,238/3,KKR,Ajinkya Rahane,...,LSG,LSG won by 4 runs,Nicholas Pooran,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://documents.iplt20.com/ipl/KKR/Logos/Log...,Lucknow Super Giants,Kolkata Knight Riders,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,T204077,"Apr 7, 2025",RCB Vs MI,"20th Match (N), Wankhede, April 07, 2025,India...","Mumbai Indians, elected to field first",RCB,Rajat Patidar,221/5,MI,Hardik Pandya,...,RCB,RCB won by 12 runs,Rajat Patidar,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Royal Challengers Bengaluru,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...


In [62]:
# batting

print(batting_summary_df.shape)
batting_summary_df.head()

(393, 12)


,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
0,T201958,RCB Vs DC,RCB,1,Phil Salt,37,17,4,3,217.64,1st Innings,out
1,T201958,RCB Vs DC,RCB,2,Virat Kohli,22,14,1,2,157.14,1st Innings,out
2,T201958,RCB Vs DC,RCB,3,Devdutt Padikkal,1,8,0,0,12.50,1st Innings,out
3,T201958,RCB Vs DC,RCB,4,Rajat Patidar,25,23,1,1,108.69,1st Innings,out
4,T201958,RCB Vs DC,RCB,5,Liam Livingstone,4,6,0,0,66.66,1st Innings,out


In [63]:
# bowling

print(bowling_summary_df.shape)
bowling_summary_df.head()

(286, 15)


,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
0,T201958,RCB Vs DC,DC,Mitchell Starc,3,0,35,0,11.66,9,3,2,1,1,1st Innings
1,T201958,RCB Vs DC,DC,Axar Patel,4,0,52,0,13.00,8,3,5,1,0,1st Innings
2,T201958,RCB Vs DC,DC,Vipraj Nigam,4,0,18,2,4.50,12,0,1,0,0,1st Innings
3,T201958,RCB Vs DC,DC,Mukesh Kumar,3,1,26,1,8.66,10,1,2,3,0,1st Innings
4,T201958,RCB Vs DC,DC,Kuldeep Yadav,4,0,17,2,4.25,11,1,0,0,0,1st Innings


In [64]:
# players data

print(players_info_df_final.shape)
players_info_df_final.head()

(227, 7)


,name,team,link,battingStyle,bowlingStyle,playingRole,image
0,Heinrich Klaasen,SRH,https://www.espncricinfo.com//cricketers/heinr...,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,Pat Cummins,SRH,https://www.espncricinfo.com//cricketers/pat-c...,Right hand Bat,Right arm Fast,Bowler,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,Abhishek Sharma,SRH,https://www.espncricinfo.com//cricketers/abhis...,Left hand Bat,Slow Left arm Orthodox,Batting Allrounder,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,Travis Head,SRH,https://www.espncricinfo.com//cricketers/travi...,Left hand Bat,Right arm Offbreak,Top order Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,Ishan Kishan,SRH,https://www.espncricinfo.com//cricketers/ishan...,Left hand Bat,Legbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...


### Save all Files

In [65]:
folder = 'abfss://1e8640c6-dc84-47a8-b295-452f8a310af8@onelake.dfs.fabric.microsoft.com/327dd03d-fc61-4a7c-8838-35fa63ac0dd5/Files/2025/'

# # 1: Teams Data (dim_teams.csv)
# df_ipl_teams.to_csv(folder + 'dim_teams.csv', index=False)


# 2: Points Table (dim_points.csv)
df_points_table.to_csv(folder + 'dim_points_table.csv', index = False)


# 3: Match Summary (dim_match_summary.csv)
df_final_match_summary.to_csv(folder + 'dim_match_summary.csv', index = False)


# 4: Batting Summary (fact_batting_summary.csv)
batting_summary_df.to_csv(folder + 'fact_batting_summary.csv', index = False)


# 5: Bowling Summary (fact_bowling_summary.csv)
bowling_summary_df.to_csv(folder + 'fact_bowling_summary.csv', index = False)


# # 6: Players Data (dim_players.csv)
# players_info_df_final.to_csv(folder + 'dim_players.csv', index = False)